### EnergyETL

In [ ]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# read csv file
file_path = "Resources/us_potential_energy.csv"

# create the data frame 
energy_df = pd.read_csv(file_path)
energy_df

In [ ]:
# select the columns
energy_df = energy_df[["Unnamed: 0", "urbanUtilityScalePV_GWh","ruralUtilityScalePV_GWh","rooftopPV_GWh", "CSP_GWh","onshoreWind_GWh","offshoreWind_GWh", "biopowerSolid_GWh", "biopowerGaseous_GWh", "geothermalHydrothermal_GWh", "EGSGeothermal_GWh", "hydropower_GWh"]]
                     
# display the new dataframe
energy_df

In [ ]:
# rename the columns
energy_df.columns=["State", "Urban_Solar", "Rural_Solar", "Rooftop_Solar", "CSP_Solar", "Onshore_Wind", "Offshore_Wind", "Biopower_Solid", "Biopower_Gaseous", "Geothermal_Hydrothermal", "EGS_Geothermal", "Hydropower"]
energy_df

In [ ]:
# change NaN values to 0
energy_df.fillna(0, inplace=True)
energy_df

### Scrape Wikipedia for the US Census Population Estimates

In [ ]:
# Store the Wikipedia URL
wiki_url = "https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population#cite_note-5"

# Use Pandas to scrape Wikipedia page for tables
wiki_tables = pd.read_html(wiki_url)

# Grab the first table
pop_est_df = wiki_tables[0]

# Grab the State & Population est. for 2018
pop_est_df = pop_est_df[['Name', 'Population estimate, July 1, 2018[5]']]

# Rename the columns
pop_est_df = pop_est_df.rename(columns={"Name": "State", "Population estimate, July 1, 2018[5]": "Population"})

# Display the new DataFrame
pop_est_df.head()